## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-13-04-04 +0000,nyt,Uncle Nearest Whiskey Is in a Fiscal Bind,https://www.nytimes.com/2025/08/14/dining/uncl...
1,2025-08-18-13-03-07 +0000,nyt,Republican Stacy Garrity Announces Bid to Chal...,https://www.nytimes.com/2025/08/18/us/politics...
2,2025-08-18-13-00-25 +0000,nyt,Zelensky Brings Backup to the White House as T...,https://www.nytimes.com/2025/08/17/us/politics...
3,2025-08-18-13-00-00 +0000,missionlocal,Brokelele,https://missionlocal.org/2025/08/sf-photos-bro...
4,2025-08-18-13-00-00 +0000,wsj,Trump’s Job Cuts Leave a Profession Looking fo...,https://www.wsj.com/us-news/education/trumps-j...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2425/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,32
48,ukraine,20
14,zelensky,19
22,putin,14
49,meeting,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
94,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,117
38,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,110
22,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,101
14,2025-08-18-12-11-00 +0000,nyt,"Russian Attacks Kill 10 in Ukraine, Hours Befo...",https://www.nytimes.com/2025/08/18/us/russia-a...,95
40,2025-08-18-10-10-29 +0000,nyt,Zelensky Calls for ‘Lasting’ Peace in Ukraine ...,https://www.nytimes.com/2025/08/18/us/politics...,92


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
94,117,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
189,54,2025-08-17-14-39-33 +0000,nypost,Putin agreed to NATO-style US security guarant...,https://nypost.com/2025/08/17/us-news/putin-ag...
135,35,2025-08-17-21-59-53 +0000,nypost,Day care worker arrested for child abuse as to...,https://nypost.com/2025/08/17/us-news/georgia-...
136,35,2025-08-17-21-57-52 +0000,nypost,Air Canada delays restart after flight attenda...,https://nypost.com/2025/08/17/business/air-can...
137,32,2025-08-17-21-45-24 +0000,nypost,New Yorkers fighting against massive battery s...,https://nypost.com/2025/08/17/us-news/new-york...
175,30,2025-08-17-17-18-00 +0000,wsj,More National Guard troops are heading soon to...,https://www.wsj.com/politics/national-security...
12,27,2025-08-18-12-31-57 +0000,nypost,Texas ICE shooting suspects tied to secretive ...,https://nypost.com/2025/08/18/us-news/texas-ic...
190,27,2025-08-17-14-37-47 +0000,nypost,HUD launches English-only initiative for all d...,https://nypost.com/2025/08/17/us-news/hud-laun...
163,25,2025-08-17-18-52-45 +0000,nypost,Nationwide strike erupts across Israel demandi...,https://nypost.com/2025/08/17/world-news/natio...
44,23,2025-08-18-10-00-00 +0000,nypost,Top tech leaders battling for AI supremacy — w...,https://nypost.com/2025/08/18/us-news/top-tech...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
